# Implementation

---

__*What are the core data structures?*__

The core data structure is a `std::vector` of `TraceRow` objects that is owned by the `Trace` class. Each row will contain its numerical value as well as an elementary operator object which will be used to calculate the derivatives. Additionally, there will be a `std::map<std::string, double>` object owned by the `Trace` class which allows for easy access to variable values based only on their name. Each function will be represented by a different instance of `Trace`, and the `Differentiator` class will own a `std::vector<Trace>` which contains each of the function traces. 

---

__*What classes will you implement?*__

Our implementation will be based on the following classes.

__`Differentiator`__ $-$ The outer class and also the public interface exposed by our library. There will be two ways to load data into the differentiator. The first option is for the differentiator to use a private instance of the `Parser` class to read the user-specified functions from a plain text file and construct the `Trace` object. Alterntively, the user can manually input functions into the differentiator object. Once the data is loaded, the trace is executed, and the `Differentiator` will return the output (a `std::vector` with either the scalar or vector output, depending on the configuration).

__`Parser`__ $-$ The class resposible for parsing the functions passed as strings into the differentiator. Each function will be a separate string, and each elementary operation will be wrapped in a pair of parentheses. For example the function, $f(x,y) = xy+x^2$ will map to the following input `((x*y)+(x^2))`. 

__`Trace`__ $-$ The main data structure which will be composed of a number of `TraceRow` elements. This class will maintain the state of the trace table as execution takes place. Once the execution is complete, the output variables will be returned to the `Differentiator` class.

__`TraceRow`__ $-$ This class represents a single row in the trace table.

__`Operator`__ $-$ This abstract class will define the interface for each elementary operator.

---
__*What method and name attributes will your classes have?*__

__`Differentiator`__ 

*Member functions*
1. `Differentiator()` $-$ Default constructor. If this is used, then `Load` must be called prior to `Run`.
2. `Differentiator(int num_inputs, int num_functions)` $-$ Overloaded constructor that specifies the number of inputs and functions for the trace.
3. `absl::Status Load(const std::string& filename)` $-$ Reads in configuration using parser. Returns status of reading.
4. `absl::StatusOr<std::vector> Run()` $-$ Runs forward mode of current configuration. Returns error status or the output in a `std::vector`.
5. `absl::Status SetVariableNames(const std::vector<char>& names)` $-$ Sets variable names according to contents of `names`.
6. `absl::Status SetFunction(const std::string& func, int i)` $-$ Sets the $i^{th}$ function to the passed in string.

*Class variables*
1. `Parser parser_` $-$ An instance of the parser class used to read configuration.
2. `std::vector<Trace> traces_` $-$ A vector holding each `Trace` corresponding to each function.

__`Parser`__

*Member functions*
1. `Parser(std::vector<char> input_names)` $-$ Constructor which obtains the names of the input variables.
2. `Trace Parse(const std::string& function)` $-$ Parses the function string based on the input variable. Returns a `Trace` object.

*Class variables*
1. `std::vector<char> input_names_` $-$ Stores the names of input variables.

__`Trace`__ 

*Member functions*
1. `Trace(int num_inputs)` $-$ Constructor which sets the number of input variables.
2. `void AddRow(TraceRow row)` $-$ Adds row to trace.
3. `double Run()` $-$ Fills in remaining numerical values of trace.

*Class variables*
1. `std::vector<TraceRow> rows_` $-$ Vector containing the trace rows.
2. `std::map<std::string, double>` $-$ Map that contains all the intermediate values of elements in the trace table.

__`TraceRow`__ 

*Member functions*
1. `TraceRow(const std::string& id, Opererator op)` $-$ Constructor to initialize id and operator.
2. `void SetValue(double val)` $-$ Sets the value of the trace node.
3. `void CalculateGradients(std::map<std::string, double> vars)` $-$ Using the map of variable and values, calcualate the gradients for each of the num inputs.


*Class variables*
1. `std::string trace_id_` $-$ The identifier of the trace variable.
2. `Operator elem_op_` $-$ The elementary operation associated with the row.
3. `double value_` $-$ The value of the node.
3. `std::vector<double> gradients_` $-$ A vector of size `num_inputs` which will contain the value of the gradients of each input variable at the given step.


__`Operator`__ 

*Member functions*
1. `Operator(std::pair<double, double> values, std::pair<double, double>)` $-$ Initialize value(s) and derivative(s) of operand(s). This must be either a single value (e.g., for trig functions) or two values (e.g., for addition).
2. `double GetValue()` $-$ Gets the value of the operand.
3. `double GetDerivative()` $-$ Gets the value of the derivative of the operator.

---

__*What external dependencies will you rely on?*__

1. C++ Standard Template Library (STL).
2. GoogleTest (`gtest`)
3. Abseil (`absl`)
4. Make/Cmake.

---

__*How will you deal with elementary functions like sin, sqrt, log, and exp?*__

The actual calculations of the elementary functions will be done using the `cmath` library. From an implementation standpoint, each operator will inherit from the `Operator` abstract class. Each of the elementary functions will then implement the `GetDerivative` interface to get numerical results based on the current values of the trace variables.